
# Created on Tue Nov 29 17:49:45 2022

# Author: Sameer Sheriff and José CAROLA DOS SANTOS

In the location of this Jupyter file, create a folder, CSMWorkdir with the following:
1. CSMBaseModel.csm
2. FAST-OAD XML Input file (inputFAST.xml or CeRAS01_baseline.xml here)
3. ESP.bat - the batch file to run ESP 

Additionally, ensure the ESP.bat file has been correctly filled as per the documentation. 

In [58]:
!pip install xmltodict
!pip install steputils
!pip install pythreejs
!pip install pyvista
!pip install stl

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for stl: filename=stl-0.0.3-py3-none-any.whl size=5347 sha256=8e606c2247c56a55bf1f951b351d5e368da3532bf5e5f26f19027bfa4feb72b4
  Stored in directory: c:\users\eric\appdata\local\pip\cache\wheels\c6\b2\c0\a33f54fbfffde82a668e4c8483af3884f91acb6b1e71b5e3d4
Successfully built stl


In [60]:
import xml.etree.ElementTree as ET
import collections #used for ordered dict
import json
import xmltodict
import os
import subprocess
import numpy
import easygui
import shutil
import pythreejs as three
from steputils import p21
import pyvista as pv
from stl import mesh
import stl2numpy

Base_folder = "Base files"
WORK_FOLDER_PATH = "CSMworkdir"

# Check if the destination folder already exists
if os.path.exists(WORK_FOLDER_PATH):
    # If it does, delete the existing folder and its contents
    shutil.rmtree(WORK_FOLDER_PATH)
# Copies contents of base file to new work folder 
shutil.copytree(Base_folder, WORK_FOLDER_PATH)

csm_file = os.path.join(WORK_FOLDER_PATH, "CsmBaseModel.csm")
batch_file = os.path.join(WORK_FOLDER_PATH, "ESP.bat")

ImportError: cannot import name 'mesh' from 'stl' (C:\Users\ERIC\anaconda3\envs\FAST-OAD\lib\site-packages\stl\__init__.py)

In [33]:
print(batch_file)
!python --version

CSMworkdir\ESP.bat
Python 3.9.16


## Copy input fast-oad file into "workdir" folder for jupyter to access

In [34]:
xml_data_file = os.path.join(WORK_FOLDER_PATH, "inputFAST.xml")
print(xml_data_file)

CSMworkdir\inputFAST.xml


This section creates odered dictionary of entire XML file

In [35]:
# Opening the xml file in read mode
with open(xml_data_file,"r") as xml_obj:
    #coverting the xml data to a Python ordered dictionary
    data_dict_ordered = xmltodict.parse(xml_obj.read())
    #closing the file
    xml_obj.close()
    #print(data_dict_ordered)
    
#converts nested ordered dict to dict
data_dict = {}
data_dict = json.loads(json.dumps(data_dict_ordered))
#print(data_dict)
#filters out the necessary geometry data values
geometry_dict = data_dict['FASTOAD_model']['data']['geometry']
print(geometry_dict)
area = geometry_dict['aircraft']['wetted_area']

{'has_T_tail': {'@is_input': 'True', '#text': '0.0'}, 'aircraft': {'wetted_area': {'@units': 'm**2', '@is_input': 'False', '#text': '834.8572353506464'}}, 'cabin': {'NPAX1': {'@is_input': 'False', '#text': '157.0'}, 'aisle_width': {'@units': 'm', '@is_input': 'True', '#text': '0.48'}, 'exit_width': {'@units': 'm', '@is_input': 'True', '#text': '0.51'}, 'length': {'@units': 'm', '@is_input': 'False', '#text': '30.380964840000004'}, 'containers': {'count_by_row': {'@is_input': 'True', '#text': '1.0'}}, 'crew_count': {'commercial': {'@is_input': 'False', '#text': '4.0'}, 'technical': {'@is_input': 'True', '#text': '2.0'}}, 'seats': {'economical': {'count_by_row': {'@is_input': 'True', '#text': '6.0'}, 'length': {'@units': 'm', '@is_input': 'True', '#text': '0.86'}, 'width': {'@units': 'm', '@is_input': 'True', '#text': '0.46'}}}}, 'flap': {'chord_ratio': {'@is_input': 'True', '#text': '0.197'}, 'span_ratio': {'@is_input': 'True', '#text': '0.8'}}, 'fuselage': {'PAX_length': {'@units': 'm'

Function to convert all entries in dict to variables 

In [36]:
xml_parameters = {}
def update_globals(d, namespace):
    for k, v in d.items():
        if 'is_input' in k or '@units' in k: #removes unwanted variables
            continue

        if isinstance(v, dict): #this line checks if v is a dictionary
            update_globals(v, namespace + k + '_')
        else:
            # This assigns each entry of the dict to a global variable 
            # This also removes the _#text from the end of the variable names
            #globals()[(namespace + k).removesuffix('_#text')] = round(float(v),5)
            xml_parameters[(namespace + k).removesuffix('_#text')] = round(float(v),5)
            #xml_parameters[f"{namespace}{k[:-5]}"] = round(float(v),5)

Call the Reclusive function

In [37]:
update_globals(geometry_dict, '')

print(xml_parameters)

{'has_T_tail': 0.0, 'aircraft_wetted_area': 834.85724, 'cabin_NPAX1': 157.0, 'cabin_aisle_width': 0.48, 'cabin_exit_width': 0.51, 'cabin_length': 30.38096, 'cabin_containers_count_by_row': 1.0, 'cabin_crew_count_commercial': 4.0, 'cabin_crew_count_technical': 2.0, 'cabin_seats_economical_count_by_row': 6.0, 'cabin_seats_economical_length': 0.86, 'cabin_seats_economical_width': 0.46, 'flap_chord_ratio': 0.197, 'flap_span_ratio': 0.8, 'fuselage_PAX_length': 22.87, 'fuselage_front_length': 6.9018, 'fuselage_length': 37.50736, 'fuselage_maximum_height': 4.05988, 'fuselage_maximum_width': 3.91988, 'fuselage_rear_length': 14.61557, 'fuselage_wetted_area': 401.956, 'horizontal_tail_area': 36.21418, 'horizontal_tail_aspect_ratio': 4.28778, 'horizontal_tail_span': 56.46108, 'horizontal_tail_sweep_0': 33.31651, 'horizontal_tail_sweep_100': 8.80894, 'horizontal_tail_sweep_25': 28.0, 'horizontal_tail_taper_ratio': 0.3, 'horizontal_tail_thickness_ratio': 0.1, 'horizontal_tail_wetted_area': 72.42837

Open csm file and update values using input XML file

In [38]:
data_file = []
with open(csm_file, 'r') as data:
    for line in data.readlines():
        data_file.append(line)       
    data.close()

# List of CSM variable names - XML variable names
variables = {#'test':'test',
            'fus:height':'fuselage_maximum_height',
            'fus:width':'fuselage_maximum_width',
            'fus:length':'fuselage_length',
            'fus:front_length':'fuselage_front_length',
            'fus:rear_length':'fuselage_rear_length',
            'fus:width':'fuselage_maximum_width',
            'fus:length':'fuselage_length',
            'wing:mac:Xat25':'wing_MAC_at25percent_x',
            'wing:mac:leadingX':'wing_MAC_leading_edge_x_local',
            'wing:mac:length':'wing_MAC_length',
            'wing:kink:leadingX':'wing_kink_leading_edge_x_local',
            'wing:kink:y':'wing_kink_y',
            'wing:tip:leadingX':'wing_tip_leading_edge_x_local',
            'wing:tip:y':'wing_tip_y',
            'vtp:mac25:Xat25':'vertical_tail_MAC_at25percent_x_local',
            'vtp:mac25:wingMac':'vertical_tail_MAC_at25percent_x_from_wingMAC25',
            'vtp:sweep:0':'vertical_tail_sweep_0',
            'vtp:span':'vertical_tail_span',
            'htp:mac25:Xat25':'horizontal_tail_MAC_at25percent_x_local',
            'htp:mac25:wingMac':'horizontal_tail_MAC_at25percent_x_from_wingMAC25',
            'htp:sweep:0':'horizontal_tail_sweep_0',
            'htp:span':'horizontal_tail_span',
            #'wing:root:Y':,#
            #'wing:root:Z':'wing_root_z',#
            'wing:root:c':'wing_root_chord',
            'wing:root:t':'wing_root_thickness_ratio',
            #'wing:root:m':'wing_root_camber',#
            #'wing:root:a':'wing_root_aoa',#
            #'wing:kink:Y':,#
            'wing:kink:c':'wing_kink_chord',
            'wing:kink:t':'wing_kink_thickness_ratio',
            #'wing:kink:m':'wing_kink_camber',#
            #'wing:kink:a':'wing_kink_aoa',#
            'wing:tip:c':'wing_tip_chord',
            'wing:tip:t':'wing_tip_thickness_ratio',
            #'wing:tip:m':'wing_tip_camber',#
            #'wing:tip:a':'wing_tip_aoa',#
            'htp:root:c':'horizontal_tail_root_chord',
            'htp:root:t':'horizontal_tail_thickness_ratio',
            #'htp:root:m':'',#
            #'htp:root:a':'',#
            'htp:tip:c':'horizontal_tail_tip_chord',
            'htp:tip:t':'horizontal_tail_thickness_ratio',
            #'htp:tip:m':'',#
            #'htp:tip:a':'',#
            'vtp:root:c':'vertical_tail_root_chord',
            'vtp:root:t':'vertical_tail_thickness_ratio',
            #'vtp:root:m':'',#
            #'vtp:root:a':'',#
            'vtp:tip:c':'vertical_tail_tip_chord',
            'vtp:tip:t':'vertical_tail_thickness_ratio',
            #'vtp:tip:m':'',#
            #'vtp:tip:a':'',#
            }

# Make a dict with updated values 
csm_values = {}
for i,j in variables.items():
    #print(j)
    csm_values[i] = xml_parameters[j]
    if j in xml_parameters:
        print(xml_parameters[j])
        
    else:
        continue
    
#print(variables)

print(csm_values)



4.05988
3.91988
37.50736
6.9018
14.61557
17.30005
3.0816
3.87253
3.14777
8.41424
9.98716
22.43797
2.71632
15.70643
40.5148
7.74816
1.68011
16.83165
33.31651
56.46108
6.05591
0.15746
2.90814
0.11937
1.16182
0.10921
4.47105
0.1
1.34132
0.1
6.83255
0.1
2.04977
0.1
{'fus:height': 4.05988, 'fus:width': 3.91988, 'fus:length': 37.50736, 'fus:front_length': 6.9018, 'fus:rear_length': 14.61557, 'wing:mac:Xat25': 17.30005, 'wing:mac:leadingX': 3.0816, 'wing:mac:length': 3.87253, 'wing:kink:leadingX': 3.14777, 'wing:kink:y': 8.41424, 'wing:tip:leadingX': 9.98716, 'wing:tip:y': 22.43797, 'vtp:mac25:Xat25': 2.71632, 'vtp:mac25:wingMac': 15.70643, 'vtp:sweep:0': 40.5148, 'vtp:span': 7.74816, 'htp:mac25:Xat25': 1.68011, 'htp:mac25:wingMac': 16.83165, 'htp:sweep:0': 33.31651, 'htp:span': 56.46108, 'wing:root:c': 6.05591, 'wing:root:t': 0.15746, 'wing:kink:c': 2.90814, 'wing:kink:t': 0.11937, 'wing:tip:c': 1.16182, 'wing:tip:t': 0.10921, 'htp:root:c': 4.47105, 'htp:root:t': 0.1, 'htp:tip:c': 1.34132, '

Function to create a new CSM file with updated values 

In [39]:
def updateCSM(data_file, csm_file):
    count = 0    
    tempString = ''
    val = 0
    des = 'DESPMTR'

    while count<len(data_file):
        tempString = data_file[count].split() #splits line with search word
        for i in csm_values:
            if des in data_file[count] and (i in data_file[count]):
                val = tempString[tempString.index(i)+1] #finds next word of search word, i.e, the value
                data_file[count] = data_file[count].replace(val, str(csm_values[i])) #replaces the value with chosen value
                #If value was found and replaced, search is ended
                break
            else:
                continue
        count+=1  
    # find old file name and add 'updated' 
    new_file = os.path.basename(csm_file)
    new_file = ('Updated_'+ new_file)
    new_file_path = os.path.join(WORK_FOLDER_PATH, new_file)
    print(new_file_path)
    # Creating a new .csm file with changes
    # opening file in 'w' or write mode
    with open(new_file_path, 'w') as x:
        for i in data_file:
            x.write(i)
        
        x.close() 
    return new_file

In [40]:
new_file = updateCSM(data_file, csm_file)
print(new_file)

CSMworkdir\Updated_CsmBaseModel.csm
Updated_CsmBaseModel.csm


# Editing batch file

In [41]:
#editing ESP run batch file
# opening file in 'r' or read mode
batch_file_data = []
with open(batch_file, 'r') as x:
    for line in x.readlines():
        batch_file_data.append(line)
    x.close()
batch_file_path = (os.getcwd()+ "\\" + WORK_FOLDER_PATH)
count = 0
while count<len(batch_file_data):
        if 'cd /d ' in batch_file_data[count]:
            batch_file_data[count] = 'cd /d ' + batch_file_path
        elif 'serveCSM' in batch_file_data[count]:
            batch_file_data[count] = '\nserveCSM "' + new_file + '"'
        # else:
        #     data_file[len(data_file)-1] = 'cd /d ' + new_file_path
        #     data_file.append('serveCSM "' + new_file + '"')

        count+=1

        
with open(batch_file, 'w') as x:
    for i in batch_file_data:
        x.write(i)
    x.close()    


#  Run batch file to model aircraft

In [42]:
%%capture captured_output
!CSMworkdir\ESP.bat > CSMworkdir\esp_logs.txt


Create a new folder 'Run _' for each new run 

In [ ]:
# Specify the prefix for the folder name
prefix = 'run'

# Initialize the counter variable file
counter_file = 'counter_file.txt'
if os.path.exists(counter_file):
    with open(counter_file, 'r') as f:
        counter = int(f.read())
else:
    counter = 0

# Increment the counter variable
counter += 1

# Create the new folder with the serialized name
run_folder = f'{prefix} {counter}'
shutil.copytree(WORK_FOLDER_PATH, run_folder)

# Save the updated counter variable to the file
with open(counter_file, 'w') as f:
    f.write(str(counter))
# Copy all the latest files to the new Run folder

